In [1]:
import os
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

from dataset.OnlineDataset import OnlineDataset
import torch

In [2]:
building_ids=[13, 20, 33, 35, 74, 75, 82, 87, 88, 101, 106, 109, 130, 144, 152, 153, 157, 161, 169, 176]

# WO Battery

In [3]:
def wo_battery(ds):
	cost = 0
	curr_ds = ds[0]
	prosumption = curr_ds['prosumption']
	price = curr_ds['price']
	for j in range(1344):

		cost  += prosumption[j] * price[j] if prosumption[j]>=0 else prosumption[j]*0.1
	return cost

# Gurobi Optimizer

In [4]:
def opti(ds):
	final_cost = 0.0
	# for i in range(51):
	curr_ds = ds[0]
	prosumption = curr_ds['prosumption']
	price = curr_ds['price']
	T=1344
	max_charge = max_power/2
	max_discharge = -max_power/2
	big_M = 1e4  # A sufficiently large number

	# Create model
	model = gp.Model("BatteryCharging")
	model.Params.OutputFlag = 0

	# Variables
	action = model.addVars(T, lb=max_discharge.item(), ub=max_charge.item(), name="action")
	soe = model.addVars(T + 1, lb=0.0, ub=battery_capacity.item(), name="soe")
	z = model.addVars(T, lb=-1.0, ub=5.0, vtype=GRB.CONTINUOUS, name='actual_price')
	b = model.addVars(T, vtype=GRB.BINARY, name="b")

	# Initial SoE
	model.addConstr(soe[0] == 0.0, name="initial_soe")

	eps = 0.0001
	M = 200 + eps

	for t in range(T):
		# Battery dynamics
		model.addConstr(soe[t + 1] == soe[t] + action[t], name=f"soe_update_{t}")
		model.addConstr(action[t]+prosumption[t] >= 0 + eps - M * (1 - b[t]), name="bigM_constr1")
		model.addConstr(action[t]+prosumption[t] <= 0 + M * b[t], name="bigM_constr2")

		# Add indicator constraints
		model.addConstr((b[t] == 1) >> (z[t] == price[t]), name="indicator_constr1")
		model.addConstr((b[t] == 0) >> (z[t] == 0.1), name="indicator_constr2")



	# Objective function: minimize cost
	obj = gp.quicksum(
		(prosumption[t] + action[t]) * z[t]
		for t in range(T)
	)
	model.setObjective(obj, GRB.MINIMIZE)

	# Optimize
	model.optimize()

	if model.status == GRB.OPTIMAL:
		actions = [action[t].X for t in range(T)]
		soes = [soe[t].X for t in range(T+1)]
		used_price = [z[t].X for t in range(T)]
		for t in range(T):
			curr_price = price[t] if (prosumption[t]+actions[t]) >= 0 else 0.1
			final_cost += (prosumption[t]+actions[t])*curr_price

	return final_cost, soes

In [5]:
# for b_id in building_ids:
for b_id in [13]:
	ds = OnlineDataset(raw_data_path='../data/1_processed',
					sliding_window_size=1344,
					sliding_window_offset=1344,
					forecast_size=0,
					building_id=b_id,
					mode='test',
					device='cpu')
	battery_capacity = ds.getBatteryCapacity()
	max_power = ds.getBatteryCapacity()/4
	print(f'Battery capacity: {battery_capacity} kWh, Max power: {max_power} kW')
	print(f'Cost without battery: {wo_battery(ds)}')
	milp, soes = opti(ds)
	print(f'Optimal cost: {milp}')

Battery capacity: 6.0 kWh, Max power: 1.5 kW
Cost without battery: 230.64524841308594
Set parameter Username
Set parameter LicenseID to value 2650516
Academic license - for non-commercial use only - expires 2026-04-11
Optimal cost: 181.72152709960938


In [6]:
import plotly.express as px
fig = px.line(soes, title=f'Customer 13')
fig.update_layout(
	yaxis_title="Power [kW]",
	template="plotly_white",
	width=600,
	margin=dict(l=10, r=10, t=30, b=10),
	xaxis=dict(dtick=6),
	legend_title_text=""
)
fig.show()

In [7]:
fig = px.line(price, title=f'Customer 13')
fig.update_layout(
	yaxis_title="Power [kW]",
	template="plotly_white",
	width=600,
	margin=dict(l=10, r=10, t=30, b=10),
	xaxis=dict(dtick=6),
	legend_title_text=""
)
fig.show()

NameError: name 'price' is not defined